In [3]:
import price_data as price
import chart
import pandas as pd
import numpy as np
import ccxt
from datetime import datetime
from users import users
from ftx_client import FtxClient
import math

ftx_ccxt = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True
})
cex= ccxt.cex({
    'uid' : 'up109520414',
    'apiKey': '1X2uEcPlvBCe4CcMtzWKguG1SDI',
    'secret': '8JY4fDg6hRz0DTolZHz77XPdC1o',
    'enableRateLimit': True,
    })

ftx_ha_trader = ccxt.ftx({
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
})

ftx_ha_trader.headers = { 'FTX-SUBACCOUNT':'HA_Trader' }
main=FtxClient(api_key='mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',api_secret='oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B')
# ftx_ccxt.headers = {'FTX-SUBACCOUNT':'Savings'}

def get_limit(symbol, markets):
    return float(next(filter(lambda x:x['symbol']==symbol, markets))['limits']['amount']['min'])


In [17]:
request = {
    'showAvgPrice': True,
}
response = ftx_ccxt.privateGetPositions(ftx_ccxt.extend(request))
result = ftx_ccxt.safe_value(response, 'result', [])
results = []
for i in range(0, len(result)):
    print(result[i])
    print(ftx_ccxt.parse_position(result[i]))
    results.append(ftx_ccxt.parse_position(result[i]))
results

{'future': 'FTT-PERP', 'size': '0.0', 'side': 'buy', 'netSize': '0.0', 'longOrderSize': '0.0', 'shortOrderSize': '0.0', 'cost': '0.0', 'entryPrice': None, 'unrealizedPnl': '0.0', 'realizedPnl': '-3.00812554', 'initialMarginRequirement': '1.0', 'maintenanceMarginRequirement': '0.03', 'openSize': '0.0', 'collateralUsed': '0.0', 'estimatedLiquidationPrice': None, 'recentAverageOpenPrice': None, 'recentPnl': None, 'recentBreakEvenPrice': None, 'cumulativeBuySize': None, 'cumulativeSellSize': None}
{'info': {'future': 'FTT-PERP', 'size': '0.0', 'side': 'buy', 'netSize': '0.0', 'longOrderSize': '0.0', 'shortOrderSize': '0.0', 'cost': '0.0', 'entryPrice': None, 'unrealizedPnl': '0.0', 'realizedPnl': '-3.00812554', 'initialMarginRequirement': '1.0', 'maintenanceMarginRequirement': '0.03', 'openSize': '0.0', 'collateralUsed': '0.0', 'estimatedLiquidationPrice': None, 'recentAverageOpenPrice': None, 'recentPnl': None, 'recentBreakEvenPrice': None, 'cumulativeBuySize': None, 'cumulativeSellSize':

[{'info': {'future': 'FTT-PERP',
   'size': '0.0',
   'side': 'buy',
   'netSize': '0.0',
   'longOrderSize': '0.0',
   'shortOrderSize': '0.0',
   'cost': '0.0',
   'entryPrice': None,
   'unrealizedPnl': '0.0',
   'realizedPnl': '-3.00812554',
   'initialMarginRequirement': '1.0',
   'maintenanceMarginRequirement': '0.03',
   'openSize': '0.0',
   'collateralUsed': '0.0',
   'estimatedLiquidationPrice': None,
   'recentAverageOpenPrice': None,
   'recentPnl': None,
   'recentBreakEvenPrice': None,
   'cumulativeBuySize': None,
   'cumulativeSellSize': None},
  'symbol': 'FTT-PERP',
  'timestamp': None,
  'datetime': None,
  'initialMargin': None,
  'initialMarginPercentage': 1.0,
  'maintenanceMargin': None,
  'maintenanceMarginPercentage': 0.03,
  'entryPrice': None,
  'notional': None,
  'leverage': 1,
  'unrealizedPnl': None,
  'contracts': 0.0,
  'contractSize': 1.0,
  'marginRatio': None,
  'liquidationPrice': None,
  'markPrice': None,
  'collateral': None,
  'marginType': 'cro

In [5]:
noDecimals=np.absolute(np.log10(next(filter(lambda x:x['symbol']=='MATIC/USD', ftx_ccxt.fetch_markets()))['precision']['amount']))

In [6]:
rob = ccxt.ftx(users['rob']['exchanges']['ftx']['api'])
if 'header' in users['rob']['exchanges']['ftx']:
    rob.headers = users['rob']['exchanges']['ftx']['header']
    print(rob.fetchPartialBalance('BTC'))

{'free': 0.01889786, 'used': 0.0, 'total': 0.01889786}


In [7]:
def transfer_ftx_to_cex(ftx_dict: dict,cex: dict,usd_amount: float,headers: dict=None, blockchain = 'SOL'):
    ftx = ccxt.ftx(ftx_dict)
    ftx.password=ftx_dict['password']
    cex = ccxt.cex(cex)
    if headers != None:
        ftx.headers = headers
    symbol=blockchain+'/'+'USD'
    current_price=price.get_current_price(symbol)
    amount = usd_amount/current_price
    noDecimals=int(np.absolute(np.log10(next(filter(lambda x:x['symbol']==symbol, ftx.fetch_markets()))['precision']['amount'])))
    amount_to_buy = round(amount,noDecimals)
    deposit_address = cex.fetchDepositAddress(blockchain)
    # ftx.createMarketBuyOrder(symbol='SOL/USD', amount=amount_to_buy)
    ftx.withdraw(code=blockchain, amount=amount_to_buy, address=deposit_address['address'])
    

my_ftx = {
    'apiKey': 'mFRyLR4AAhLTc5RlWov3PKTcIbMHw3vGZwiHnsrn',
    'secret': 'oKaY1WEqTuhnNnq0iRi_Ry-CYckvE89-gPUPf21B',
    'enableRateLimit': True,
    'password': 'Ev443978??'
}

my_cex = {
    'uid' : 'up109520414',
    'apiKey': '1X2uEcPlvBCe4CcMtzWKguG1SDI',
    'secret': '8JY4fDg6hRz0DTolZHz77XPdC1o',
    'enableRateLimit': True,
}

my_savings = {'FTX-SUBACCOUNT':'Savings'}
# transfer_ftx_to_cex(my_ftx,my_cex,50, my_savings)
edward = ccxt.ftx(users['edward']['exchanges']['ftx']['api'])
edward.fetch_partial_balance('USD')


{'free': 4.46752782, 'used': 0.0, 'total': 4.46752782}

In [8]:
float(ftx_ha_trader.fetch_balance()['USD']['total'])

1064.25491167

In [9]:
main.get_position('ETH-PERP', True)

{'future': 'ETH-PERP',
 'size': 0.404,
 'side': 'sell',
 'netSize': -0.404,
 'longOrderSize': 0.404,
 'shortOrderSize': 0.0,
 'cost': -1038.4416,
 'entryPrice': 2570.4,
 'unrealizedPnl': 0.0,
 'realizedPnl': -105.29560443,
 'initialMarginRequirement': 1.0,
 'maintenanceMarginRequirement': 0.03,
 'openSize': 0.404,
 'collateralUsed': 1038.4416,
 'estimatedLiquidationPrice': 5117.041632956436,
 'recentAverageOpenPrice': 2635.6,
 'recentPnl': 41.5483,
 'recentBreakEvenPrice': 2673.242326732673,
 'cumulativeBuySize': 0.385,
 'cumulativeSellSize': 0.789}